In [107]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [108]:


%set_env CUDA_LAUNCH_BLOCKING=1



env: CUDA_LAUNCH_BLOCKING=1


In [109]:
import sys
import cv2
import os
import math
import random
import pickle
import copy
import numpy as np
import pandas as pd
sys.path.append("C:github/MeronymNet-PyTorch/src")

In [110]:
import torch
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import DataLoader
import torch.utils.data as data_utils
from torch.utils.tensorboard import SummaryWriter

In [111]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


In [112]:
%cd ..
%cd ..
%cd ..
%cd GitHub
%cd MeronymNet-PyTorch/src


C:\GitHub\MeronymNet-PyTorch
C:\GitHub
C:\
C:\GitHub
C:\GitHub\MeronymNet-PyTorch\src


In [113]:
from losses import BoxVAE_losses as loss
from evaluation import metrics
from utils import plot_utils
from components.AutoEncoder import GCNAutoEncoder
from components.Decoder import Decoder

In [114]:
import matplotlib.pyplot as plt

In [115]:
# from mask_generation import masked_sketch

In [116]:
colors = [(1, 0, 0),
          (0.737, 0.561, 0.561),
          (0.255, 0.412, 0.882),
          (0.545, 0.271, 0.0745),
          (0.98, 0.502, 0.447),
          (0.98, 0.643, 0.376),
          (0.18, 0.545, 0.341),
          (0.502, 0, 0.502),
          (0.627, 0.322, 0.176),
          (0.753, 0.753, 0.753),
          (0.529, 0.808, 0.922),
          (0.416, 0.353, 0.804),
          (0.439, 0.502, 0.565),
          (0.784, 0.302, 0.565),
          (0.867, 0.627, 0.867),
          (0, 1, 0.498),
          (0.275, 0.51, 0.706),
          (0.824, 0.706, 0.549),
          (0, 0.502, 0.502),
          (0.847, 0.749, 0.847),
          (1, 0.388, 0.278),
          (0.251, 0.878, 0.816),
          (0.933, 0.51, 0.933),
          (0.961, 0.871, 0.702)]
colors = (np.asarray(colors)*255)
canvas_size = 660

def plot_bbx(bbx):
    bbx = bbx*canvas_size
    canvas = np.ones((canvas_size,canvas_size,3), np.uint8) * 255
    for i, coord in enumerate(bbx):
        x_minp, y_minp,x_maxp , y_maxp= coord
        if [x_minp, y_minp,x_maxp , y_maxp]!=[0,0,0,0]:
            cv2.rectangle(canvas, (int(x_minp), int(y_minp)), (int(x_maxp) , int(y_maxp) ), colors[i], 6)
    return canvas

# def plot_bbx(bbx):
#     bbx = bbx*canvas_size
#     canvas = np.ones((canvas_size,canvas_size,3), np.uint8) * 255
#     for i, coord in enumerate(bbx):
#         x, y, w ,h = coord
#         if [x, y, w ,h]!=[0,0,0,0]:
#             cv2.rectangle(canvas, (int(x), int(y)), (int(x + w) , int(y + h) ), colors[i], 6)
#     return canvas


In [117]:
def inference(decoder, nodes, obj_class, latent_dims, batch_size):

    decoder.cuda()
    z_latent = torch.normal(torch.zeros([batch_size,latent_dims])).cuda()
    nodes = torch.reshape(nodes,(batch_size,decoder.num_nodes))
    obj_class = torch.reshape(obj_class,(batch_size, decoder.class_size))
    conditioned_z = torch.cat([nodes, z_latent],dim=-1)
    conditioned_z = torch.cat([obj_class, conditioned_z],dim=-1)
    
    x_bbx, x_lbl, x_edge, class_pred = decoder(conditioned_z)
    
    return x_bbx, x_lbl, z_latent

In [118]:
"C:\GitHub\meronymnet\data_np_16"

'C:\\GitHub\\meronymnet\\data_np_16'

In [101]:
directory = "C:/GitHub/meronymnet/data_np_16"

outfile = os.path.join(directory, "X_train_combined.np")
with open(outfile, 'rb') as pickle_file:
    X_train = pickle.load(pickle_file)

outfile = os.path.join(directory, "class_v_combined.np")
with open(outfile, 'rb') as pickle_file:
    class_v = pickle.load(pickle_file)

outfile = os.path.join(directory, "adj_train_combined.np")
with open(outfile, 'rb') as pickle_file:
    adj_train = pickle.load(pickle_file)

outfile = os.path.join(directory, "X_train_val_combined.np")
with open(outfile, 'rb') as pickle_file:
    X_train_val = pickle.load(pickle_file)

outfile = os.path.join(directory, "class_v_val_combined.np")
with open(outfile, 'rb') as pickle_file:
    class_v_val = pickle.load(pickle_file)

outfile = os.path.join(directory, "adj_train_val_combined.np")
with open(outfile, 'rb') as pickle_file:
    adj_train_val = pickle.load(pickle_file)

outfile = os.path.join(directory, "X_test_combined.np")
with open(outfile, 'rb') as pickle_file:
    X_test = pickle.load(pickle_file)

outfile = os.path.join(directory, "adj_test_combined.np")
with open(outfile, 'rb') as pickle_file:
    adj_test = pickle.load(pickle_file)

outfile = os.path.join(directory, "class_v_test_combined.np")
with open(outfile, 'rb') as pickle_file:
    class_v_test = pickle.load(pickle_file)


In [124]:
outfile = 'C:/GitHub/meronymnet/data_np_16/X_train_combined_mask_data_1.np'
with open(outfile, 'rb') as pickle_file:
    X_train = pickle.load(pickle_file)

outfile = 'C:/GitHub/meronymnet/data_np_16/class_v_combined_mask_data_1.np'
with open(outfile, 'rb') as pickle_file:
    class_v = pickle.load(pickle_file)

outfile = 'C:/GitHub/meronymnet/data_np_16/adj_train_combined_mask_data_1.np'
with open(outfile, 'rb') as pickle_file:
    adj_train = pickle.load(pickle_file)

outfile = 'C:/GitHub/meronymnet/data_np_16/X_train_val_combined_mask_data_1.np'
with open(outfile, 'rb') as pickle_file:
    X_train_val = pickle.load(pickle_file)

outfile = 'C:/GitHub/meronymnet/data_np_16/class_v_train_val_combined_mask_data_1.np'
with open(outfile, 'rb') as pickle_file:
    class_v_val = pickle.load(pickle_file)
    
outfile = 'C:/GitHub/meronymnet/data_np_16/adj_train_val_combined_mask_data_1.np'
with open(outfile, 'rb') as pickle_file:
    adj_train_val = pickle.load(pickle_file)
    
outfile = 'C:/GitHub/meronymnet/data_np_16/X_test_combined_mask_data_1.np'
with open(outfile, 'rb') as pickle_file:
    X_test = pickle.load(pickle_file)

outfile = 'C:/GitHub/meronymnet/data_np_16/adj_test_combined_mask_data_1.np'
with open(outfile, 'rb') as pickle_file:
    adj_test = pickle.load(pickle_file)
    
outfile = 'C:/GitHub/meronymnet/data_np_16/class_v_test_combined_mask_data_1.np'
with open(outfile, 'rb') as pickle_file:
    class_v_test = pickle.load(pickle_file)

In [125]:
X_train[X_train<=0] = 0
X_train_val[X_train_val<=0] = 0
X_test[X_test<=0] = 0

X_train[X_train>=1] = 1
X_train_val[X_train_val>=1] = 1
X_test[X_test>=1] = 1

In [126]:
print(X_train.shape)
print(X_train_val.shape)
print(X_test.shape)

(50910, 16, 5)
(6720, 16, 5)
(10380, 16, 5)


In [122]:
print(X_train.shape)
print(X_train_val.shape)
print(X_test.shape)

(3394, 16, 5)
(448, 16, 5)
(692, 16, 5)


In [127]:
random.seed(100)
train_idx = np.random.randint(1,len(X_train),len(X_train))
val_idx = np.random.randint(1,len(X_train_val),len(X_train_val))
test_idx = np.random.randint(1,len(X_test),len(X_test))

In [128]:
batch_size = 128
seed = 345

torch.manual_seed(seed)

validation = True
if validation:
    train_list =[]
    for idx, batch in enumerate(zip(copy.deepcopy(X_train[train_idx]),
                                    copy.deepcopy(class_v[train_idx]),
                                    copy.deepcopy(adj_train[train_idx]))):
        edge_index, _ = dense_to_sparse(torch.from_numpy(batch[2]).cuda().float())
        train_list.append(Data(x = torch.from_numpy(batch[0]).cuda().float(),
                               y = torch.from_numpy(batch[1]).cuda().float(),
                               edge_index = edge_index
                                    )
                         )

    batch_train_loader = DataLoader(train_list, batch_size=batch_size)

    val_list = []
    for idx, batch in enumerate(zip(copy.deepcopy(X_train_val[val_idx]),
                                    copy.deepcopy(class_v_val[val_idx]), 
                                    copy.deepcopy(adj_train_val[val_idx]))):
        edge_index, _ = dense_to_sparse(torch.from_numpy(batch[2]).cuda().float())
        val_list.append(Data(x = torch.from_numpy(batch[0]).cuda().float(),
                             y = torch.from_numpy(batch[1]).cuda().float(),
                             edge_index = edge_index
                                    )
                         )
    batch_val_loader = DataLoader(val_list, batch_size=batch_size)
else:
    train_list =[]
    for idx, batch in enumerate(zip(copy.deepcopy(X_train[train_idx]),
                                    copy.deepcopy(class_v[train_idx]),
                                    copy.deepcopy(adj_train[train_idx]))):
        edge_index, _ = dense_to_sparse(torch.from_numpy(batch[2]).cuda())
        train_list.append(Data(x = torch.from_numpy(batch[0]).cuda(),
                               y = torch.from_numpy(batch[1]).cuda(),
                               edge_index = edge_index
                                    )
                         )
    
    for idx, batch in enumerate(zip(copy.deepcopy(X_train_val[val_idx]),
                                    copy.deepcopy(class_v_val[val_idx]), 
                                    copy.deepcopy(adj_train_val[val_idx]))):
        edge_index, _ = dense_to_sparse(torch.from_numpy(batch[2]).cuda())
        train_list.append(Data(x = torch.from_numpy(batch[0]).cuda(),
                             y = torch.from_numpy(batch[1]).cuda(),
                             edge_index = edge_index
                                    )
                         )
    batch_train_loader = DataLoader(train_list, batch_size=batch_size)
    
    val_list = []
    for idx, batch in enumerate(zip(copy.deepcopy(X_test[test_idx]),
                                    copy.deepcopy(class_v_test[test_idx]), 
                                    copy.deepcopy(adj_test[test_idx]))):
        edge_index, _ = dense_to_sparse(torch.from_numpy(batch[2]).cuda())
        val_list.append(Data(x = torch.from_numpy(batch[0]).cuda(),
                             y = torch.from_numpy(batch[1]).cuda(),
                             edge_index = edge_index
                                    )
                         )
    batch_val_loader = DataLoader(val_list, batch_size=batch_size)
    

In [129]:
del train_list
del val_list
del X_train
del class_v
del adj_train
del X_train_val
del class_v_val
del adj_train_val

In [130]:
idx = 0
for data in batch_train_loader:
    idx+=1
    print(data.x.shape)
    if idx==3:
        break

torch.Size([2048, 5])
torch.Size([2048, 5])
torch.Size([2048, 5])


In [138]:
latent_dims = 64
batch_size = 128
num_nodes = 16
bbx_size = 4
num_classes = 7
label_shape = 1
nb_epochs = 300
klw = loss.frange_cycle_linear(nb_epochs)
learning_rate = 0.00008
hidden1 = 32
hidden2 = 16
hidden3 = 128
run_prefix = "base_model"

In [139]:
import gc
gc.collect()

2125

In [140]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [141]:
reconstruction_loss_arr = []
kl_loss_arr = []
bbox_loss_arr = []
adj_loss_arr = []
node_loss_arr = []

reconstruction_loss_val_arr = []
kl_loss_val_arr = []
bbox_loss_val_arr = []
adj_loss_val_arr = []
node_loss_val_arr = []

vae = GCNAutoEncoder(latent_dims,num_nodes,bbx_size,num_classes,label_shape,hidden1, hidden2, hidden3,
                 predict_edges=True,
                 predict_class=True,)
vae.to(device)
optimizer = optim.Adam(vae.parameters(), lr=learning_rate)
model_path = ('C:/GitHub/MeronymNet-PyTorch/src/Box-GCNVAE/'+run_prefix+'/GCN-lr-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('C:/GitHub/MeronymNet-PyTorch/src/Box-GCNVAE/'+run_prefix+'/GCN-lr-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
if not os.path.exists(model_path):
    os.makedirs(model_path)
writer = SummaryWriter(summary_path)
icoef = 0

In [142]:
for epoch in range(nb_epochs):

    batch_loss = torch.tensor([0.0])
    batch_kl_loss = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    batch_adj_loss = torch.tensor([0.0])
    batch_node_loss = torch.tensor([0.0])
    batch_class_loss = torch.tensor([0.0])
    images = []
    
    vae.train()
    i=0
    for train_data in batch_train_loader:
        node_data_true = train_data.x
        label_true = node_data_true[:,:1]
        class_true = train_data.y
        adj_true = train_data.edge_index
        batch = train_data.batch
        
        for param in vae.parameters():
            param.grad=None
        
        output = vae(adj_true, node_data_true, label_true, class_true, predict_adj_class=True)
        node_data_pred, label_pred, adj_pred, class_pred, z_mean, z_logvar = output
        
        kl_loss = loss.kl_loss(z_mean, z_logvar)
        adj_loss = loss.adj_loss(adj_pred, adj_true, batch, num_nodes)
        bbox_loss = loss.bbox_loss(node_data_pred, node_data_true[:,1:])
        node_loss = loss.node_loss(label_pred,label_true)
        class_loss = loss.class_loss(class_pred, class_true)
        
        kl_weight = klw[icoef]
        
        if kl_weight>0:
            reconstruction_loss = kl_loss*kl_weight + (bbox_loss + node_loss + adj_loss + class_loss)*num_nodes*5
        else:
            reconstruction_loss = (bbox_loss + node_loss)*num_nodes*5
            
        reconstruction_loss.backward()
        
        optimizer.step()
        i+=1
      
        batch_loss += reconstruction_loss
        batch_kl_loss += kl_loss
        batch_bbox_loss += bbox_loss
        batch_adj_loss += adj_loss
        batch_node_loss += node_loss
        batch_class_loss += class_loss
    
        if i%200==0:
            print(i)
            global_step = epoch*len(batch_train_loader)+i
            
            writer.add_scalar("Loss/train/reconstruction_loss", batch_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/kl_loss", batch_kl_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/bbox_loss", batch_bbox_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/adjacency_loss", batch_adj_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/node_loss", batch_node_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/class_loss", batch_class_loss.item()/(i+1), global_step)
       
    global_step = epoch*len(batch_train_loader)+i
    image_shape = [num_nodes, bbx_size]

    image = plot_bbx(np.reshape((node_data_true[:num_nodes,1:]*label_true[:num_nodes]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('train/images/input', image, global_step, dataformats='HWC')
    image = plot_bbx((node_data_pred[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('train/images/generated', image, global_step, dataformats='HWC')
    
    reconstruction_loss_arr.append(batch_loss.detach().item()/(i+1))
    kl_loss_arr.append(batch_kl_loss.detach().item()/(i+1))
    bbox_loss_arr.append(batch_bbox_loss.detach().item()/(i+1))
    adj_loss_arr.append(batch_adj_loss.detach().item()/(i+1))
    node_loss_arr.append(batch_node_loss.detach().item()/(i+1))
    
    print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, batch_loss/(i+1) ))
    
    batch_loss = torch.tensor([0.0])
    batch_kl_loss = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    batch_adj_loss = torch.tensor([0.0])
    batch_node_loss = torch.tensor([0.0])
    batch_class_loss = torch.tensor([0.0])
    images = []
    vae.eval()
    for i, val_data in enumerate(batch_val_loader, 0):
        node_data_true = val_data.x
        label_true = node_data_true[:,:1]
        class_true = val_data.y
        adj_true = val_data.edge_index
        batch = val_data.batch
        
        kl_weight = klw[icoef]
        
        output = vae(adj_true, node_data_true, label_true , class_true, predict_adj_class=True)
        node_data_pred, label_pred, adj_pred, class_pred, z_mean, z_logvar = output
        kl_loss = loss.kl_loss(z_mean, z_logvar)
        adj_loss = loss.adj_loss(adj_pred, adj_true, batch, num_nodes)
        bbox_loss = loss.bbox_loss(node_data_pred, node_data_true[:,1:])
        node_loss = loss.node_loss(label_pred,label_true)
        class_loss = loss.class_loss(class_pred, class_true)
        
        reconstruction_loss = kl_loss*kl_weight + (bbox_loss + node_loss + adj_loss + class_loss)*num_nodes*5
        
        batch_loss += reconstruction_loss
        batch_kl_loss += kl_loss
        batch_bbox_loss += bbox_loss
        batch_adj_loss += adj_loss
        batch_node_loss += node_loss
        batch_class_loss += class_loss
    
    image = plot_bbx(np.reshape((node_data_true[:num_nodes,1:]*label_true[:num_nodes]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('val/images/input', image, global_step, dataformats='HWC')
    image = plot_bbx((node_data_pred[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('val/images/generated', image, global_step, dataformats='HWC')

    reconstruction_loss_val_arr.append(batch_loss.detach().item()/(i+1))
    kl_loss_val_arr.append(batch_kl_loss.detach().item()/(i+1))
    bbox_loss_val_arr.append(batch_bbox_loss.detach().item()/(i+1))
    adj_loss_val_arr.append(batch_adj_loss.detach().item()/(i+1))
    node_loss_val_arr.append(batch_node_loss.detach().item()/(i+1))
    
    writer.add_scalar("Loss/val/reconstruction_loss", batch_loss.detach()/(i+1), global_step)
    writer.add_scalar("Loss/val/kl_loss", batch_kl_loss.detach()/(i+1), global_step)
    writer.add_scalar("Loss/val/bbox_loss", batch_bbox_loss.detach()/(i+1), global_step)
    writer.add_scalar("Loss/val/adjacency_loss", batch_adj_loss.detach()/(i+1), global_step)
    writer.add_scalar("Loss/val/node_loss", batch_node_loss.detach()/(i+1), global_step)
    writer.add_scalar("Loss/val/class_loss", batch_class_loss.detach()/(i+1), global_step)
       
    if epoch%50 == 0:
        torch.save(vae.state_dict(), model_path + '/model_weights.pth')

#     if kl_loss_arr[-1]>0.5 and abs(bbox_loss_arr[-1] - bbox_loss_val_arr[-1]) < 0.013 and bbox_loss_arr[-1]<0.12 and epoch>60:
#         icoef = icoef + 1  

torch.save(vae.state_dict(),model_path + '/model_weights.pth')

for i in range(min(100,int(len(node_data_true)/num_nodes))):    
    image = plot_bbx(np.reshape((node_data_true[num_nodes*i:num_nodes*(i+1),1:]*label_true[num_nodes*i:num_nodes*(i+1)]).detach().to("cpu").numpy(),
                                    image_shape)).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-input', image, global_step, dataformats='HWC')
    image = plot_bbx((node_data_pred[i]*label_true[num_nodes*i:num_nodes*(i+1)]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-generated', image, global_step, dataformats='HWC')
    
writer.flush()
writer.close()
print('Finished Training')

C:\GitHub\MeronymNet-PyTorch\src\components\Decoder.py:71: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  class_pred = self.act2(self.dense_cls(x))


200
[1,   399] loss: 409.903
200
[2,   399] loss: 345.796
200
[3,   399] loss: 345.080
200
[4,   399] loss: 344.290
200
[5,   399] loss: 334.578
200
[6,   399] loss: 292.359
200
[7,   399] loss: 271.268
200
[8,   399] loss: 263.333
200
[9,   399] loss: 250.914
200
[10,   399] loss: 224.787
200
[11,   399] loss: 211.793
200
[12,   399] loss: 203.415
200
[13,   399] loss: 196.038
200
[14,   399] loss: 187.751
200
[15,   399] loss: 178.077
200
[16,   399] loss: 169.554
200
[17,   399] loss: 162.349
200
[18,   399] loss: 156.878
200
[19,   399] loss: 152.775
200
[20,   399] loss: 149.470
200
[21,   399] loss: 146.630
200
[22,   399] loss: 144.078
200
[23,   399] loss: 141.727
200
[24,   399] loss: 139.526
200
[25,   399] loss: 137.438
200
[26,   399] loss: 135.417
200
[27,   399] loss: 133.445
200
[28,   399] loss: 131.509
200
[29,   399] loss: 129.589
200
[30,   399] loss: 127.642
200
[31,   399] loss: 125.607
200
[32,   399] loss: 123.413
200
[33,   399] loss: 121.066
200
[34,   399] los

In [146]:
#testing loop
model_path = ('C:/GitHub/MeronymNet-PyTorch/src/Box-GCNVAE/'+run_prefix+'/GCN-lr-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('C:/GitHub/MeronymNet-PyTorch/src/Box-GCNVAE/'+run_prefix+'/GCN-lr-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')


write_tensorboard = False
if write_tensorboard:
    writer = SummaryWriter(summary_path)

vae = GCNAutoEncoder(latent_dims,num_nodes,bbx_size,num_classes,label_shape,hidden1, hidden2, hidden3,
                 predict_edges=True,
                 predict_class=True,)
vae.load_state_dict(torch.load(model_path+ '/model_weights.pth'))

# decoder = vae.decoder
image_shape = [num_nodes, bbx_size]
global_step = 250000
class_dict = {0:'cow', 1:'sheep', 2:'bird', 3:'person', 4:'cat', 5:'dog', 6:'horse'}
res_dfs = []
for i, val_data in enumerate(batch_val_loader, 0):
    
    val_data.cuda()
    node_data_true = val_data.x
    label_true = node_data_true[:,:1]
    class_true = val_data.y
    #print(class_true.size())
    val_batch_size = int(class_true.shape[0]/10)
    #print(val_batch_size)
    adj_true = val_data.edge_index
    class_true  = torch.flatten(class_true)
    
    output = vae(adj_true, node_data_true, label_true , class_true, predict_adj_class=True)
    node_data_pred_test, label_pred, adj_pred, class_pred, z_mean, z_logvar = output
    
    res_dfs.append(metrics.get_metrics(node_data_true, None, node_data_pred_test,
                               None,
                               label_true,
                               class_true,
                               num_nodes,
                               num_classes,
                               False))
    
    if write_tensorboard:
        
        for j in range(int(len(node_data_true)/num_nodes)):
            
            image = plot_bbx(node_data_true[j].detach().to("cpu").numpy().astype(float))/255
            pred_image = plot_bbx(node_data_pred_test[j].detach().to("cpu").numpy()).astype(float)/255

            writer.add_image('test_result/images/'+str(j)+'-input/', image, global_step, dataformats='HWC')  
            writer.add_image('test_result/images/'+str(j)+'-generated/', pred_image, global_step, dataformats='HWC')
            

result = pd.concat(res_dfs)
result['obj_class'] = np.where(result['obj_class'].isna(), 0, result['obj_class'])
result['obj_class'] = result['obj_class'].astype('int')
result['obj_class'].replace(class_dict, inplace=True)
result.where(result['part_labels']!=0, np.NaN, inplace=True)
result['part_labels'] = np.where(result['part_labels'].isna(), 0, result['part_labels'])
result['part_labels'] = result['part_labels'].astype('int')
result['id'] = np.repeat(np.array(list(range(int(len(result)/num_nodes)))), num_nodes)

if write_tensorboard:
    writer.flush()
    writer.close()

bird_labels = {'head':1 , 'torso':2, 'neck':3, 'lwing':4, 'rwing':5, 'lleg':6, 'lfoot':7, 'rleg':8, 'rfoot':9, 'tail':10}
cat_labels = {'head':1, 'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12}
cow_labels = {'head':1,'lhorn':2, 'rhorn':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14}
dog_labels = {'head':1,'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12, 'muzzle':13}
horse_labels = {'head':1,'lfho':2, 'rfho':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14, 'lbho':15, 'rbho':16}
person_labels = {'head':1, 'torso':2, 'neck': 3, 'llarm': 4, 'luarm': 5, 'lhand': 6, 'rlarm':7, 'ruarm':8, 'rhand': 9, 'llleg': 10, 'luleg':11, 'lfoot':12, 'rlleg':13, 'ruleg':14, 'rfoot':15}
sheep_labels = cow_labels
part_labels_combined_parts = {'bird': bird_labels, 'cat': cat_labels, 'cow': cow_labels, 'dog': dog_labels, 'sheep': sheep_labels, 'horse':horse_labels,'person':person_labels}


for k, v in part_labels_combined_parts.items():
    new_map = {}
    for pk, pv in v.items():
        new_map[pv]=pk
    part_labels_combined_parts[k] = new_map
    
for k, v in part_labels_combined_parts.items():
    result.loc[result['obj_class']==k, ['part_labels']] = result.loc[result['obj_class']==k,['part_labels']].replace(v).copy()

result.to_csv(model_path+'/raw_metrics.csv')
print(result)
res_obj_level = result[['obj_class', 'IOU', 'MSE', 'id']].groupby(['obj_class', 'id']).mean().reset_index()
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']].to_csv(model_path+'/obj_level_metrics.csv')
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',  'IOU', 'MSE']].to_csv(model_path+'/part_level_metrics.csv')

C:\GitHub\MeronymNet-PyTorch\src\components\Decoder.py:71: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  class_pred = self.act2(self.dense_cls(x))
C:\GitHub\MeronymNet-PyTorch\src\evaluation\metrics.py:21: RuntimeWarning: invalid value encountered in true_divide
  iouk = interArea / unionArea


     obj_class part_labels       IOU       MSE    id
0          dog        head  0.646839  0.001180     0
1          dog       torso  0.654369  0.003530     0
2          dog        neck  0.379547  0.002731     0
3          dog       lfleg  0.299619  0.006268     0
4          NaN           0       NaN       NaN     0
...        ...         ...       ...       ...   ...
1019       NaN           0       NaN       NaN  6719
1020       NaN           0       NaN       NaN  6719
1021       NaN           0       NaN       NaN  6719
1022       NaN           0       NaN       NaN  6719
1023       NaN           0       NaN       NaN  6719

[107520 rows x 5 columns]


In [147]:
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']]

,obj_class,IOU,MSE
0,bird,0.435339,0.008547
1,cat,0.535627,0.006043
2,cow,0.447185,0.004416
3,dog,0.419372,0.006610
4,horse,0.352277,0.006046
5,person,0.443944,0.005871
6,sheep,0.610709,0.006018


In [148]:
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',  'IOU', 'MSE']]

,obj_class,part_labels,IOU,MSE
0,bird,head,0.488423,0.005360
1,bird,lfoot,0.169141,0.005270
2,bird,lleg,0.267321,0.004616
3,bird,lwing,0.322880,0.021939
4,bird,neck,0.376385,0.006239
...,...,...,...,...
81,sheep,lhorn,0.191333,0.014847
82,sheep,neck,0.630799,0.003208
83,sheep,rhorn,0.109285,0.015523
84,sheep,tail,0.334328,0.014160


In [149]:
#testing loop
model_path = ('C:/GitHub/MeronymNet-PyTorch/src/Box-GCNVAE/'+run_prefix+'/GCN-lr-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('C:/GitHub/MeronymNet-PyTorch/src/Box-GCNVAE/'+run_prefix+'/GCN-lr-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
write_tensorboard = True
if write_tensorboard:
    writer = SummaryWriter(summary_path)

vae = GCNAutoEncoder(latent_dims,num_nodes,bbx_size,num_classes,label_shape,hidden1, hidden2, hidden3)
vae.load_state_dict(torch.load(model_path+ '/model_weights.pth'))

decoder = vae.decoder
image_shape = [num_nodes, bbx_size]
global_step = 250000
pred_boxes = []
classes = []
for i, val_data in enumerate(batch_val_loader, 0):

    
    val_data.cuda()
    
    node_data_true = val_data.x
    #print(node_data_true.size())
    label_true = node_data_true[:,:1]
    #print(label_true.size())
    class_true = val_data.y
    #print(class_true.size())
    
    val_batch_size = int(class_true.shape[0]/7)
    #val_batch_size = 128
    #print(val_batch_size)
    output = inference(decoder, label_true , class_true, latent_dims, val_batch_size)
    node_data_pred_test, _, _ = output
    pred_boxes.append((node_data_pred_test*label_true.reshape([int(label_true.shape[0]/num_nodes),num_nodes,1])).detach().to("cpu").numpy())
    classes.append(class_true.detach().to("cpu").numpy())

if write_tensorboard:
    
    for i in range(int(len(node_data_true)/24)):    
        image = plot_bbx(np.reshape((node_data_true[num_nodes*i:num_nodes*(i+1),1:]*label_true[num_nodes*i:num_nodes*(i+1)]).detach().to("cpu").numpy(),
                                    image_shape)).astype(float)/255
        pred_image = plot_bbx((node_data_pred_test[i]*label_true[num_nodes*i:num_nodes*(i+1)]).detach().to("cpu").numpy()).astype(float)/255
        pred_boxes.append((node_data_pred_test[i]*label_true[num_nodes*i:num_nodes*(i+1)]).detach().to("cpu").numpy())

        writer.add_image('test_result/images/'+str(i)+'-input/', image, global_step, dataformats='HWC')  
        writer.add_image('test_result/images/'+str(i)+'-generated/', pred_image, global_step, dataformats='HWC')

    writer.flush()
    writer.close()
    


In [150]:
output

(tensor([[[0.1311, 0.4431, 0.2152, 0.5654],
          [0.3169, 0.2238, 0.8566, 0.5404],
          [0.3847, 0.2765, 0.4384, 0.3312],
          ...,
          [0.7173, 0.2397, 0.8841, 0.3436],
          [0.4752, 0.5267, 0.7278, 0.6516],
          [0.9090, 0.8190, 0.9432, 0.8222]],
 
         [[0.0757, 0.1880, 0.5878, 0.7789],
          [0.5820, 0.3290, 0.9348, 0.8386],
          [0.4574, 0.4519, 0.6370, 0.7279],
          ...,
          [0.8054, 0.4943, 0.8700, 0.6361],
          [0.8107, 0.6891, 0.9881, 0.5981],
          [0.9936, 0.7179, 0.9971, 0.6520]],
 
         [[0.2974, 0.1911, 0.5348, 0.6092],
          [0.2974, 0.1722, 0.8098, 0.8023],
          [0.4568, 0.2824, 0.5887, 0.6069],
          ...,
          [0.6287, 0.3304, 0.6821, 0.4147],
          [0.2968, 0.3210, 0.8883, 0.1941],
          [0.9773, 0.4791, 0.9906, 0.3664]],
 
         ...,
 
         [[0.0551, 0.2134, 0.3023, 0.4955],
          [0.2150, 0.1762, 0.9084, 0.7688],
          [0.2649, 0.2500, 0.4337, 0.4342],
      

In [151]:
#for x in pred_boxes:
#    print(len(x))
    

In [153]:
outfile = 'C:/GitHub/meronymnet/outputs_GCN_VAE/generate_boxes.npy'
with open(outfile, 'wb') as pickle_file:
    pickle.dump(pred_boxes, pickle_file)
outfile = 'C:/GitHub/meronymnet/outputs_GCN_VAE/generate_boxes_obj_class.npy'
with open(outfile, 'wb') as pickle_file:
    pickle.dump(classes,pickle_file)

In [154]:
import gc
gc.collect()

1212